# Mosaic
Inputs:
- goal image
- folder of images
- number of photos you want in image

Output
- image

In [264]:
INPUT_IMAGE_PATH = "data/targetimages/spiderman.jpg"
INPUT_MASK_PATH = "data/targetimages/mask_spiderman.jpg"
MASK_COLOR_RGB = (255,0,200)
OUTPUT_IMAGE_PATH = "data/outputimages/spidermanoutput-1.jpg"
ALBUM_FOLDER_PATH = "data/albums/spiderman/"
ALBUM_FOLDER_INDEX = "data/albums/spiderman/indexed.txt"
ALBUM_PHOTO_EXTENSION = "*.jpg"
subPhotoPixelLength = 700
numberSubPhotosHorizontal = 10


# Step 0: Run Helper Classes
# 0.1 DistanceHeuristics


In [46]:
from PIL import Image
from IPython.display import clear_output
import numpy as np

from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000

import glob
import json

In [233]:
class DistanceHeuristics:
    def __init__(self, indexPath):
        with open(indexPath, "r") as f:
            data = f.read()
            self.photos = json.loads(data)
            print("Photos index loaded.")

        self.lookup = np.ndarray((256,256,256,1), dtype="int")
    
    def deltaCIE(self, inputColor):
        color2_rgb = sRGBColor(inputColor[0], inputColor[1], inputColor[2])
        # Convert from RGB to Lab Color Space
        color2_lab = convert_color(color2_rgb, LabColor)

        delta = {
            "path": "none",
            "amount": 10000000
        }

        for photo in self.photos:
            photoRGB = eval(photo['dominantColor'])
            color1_rgb = sRGBColor(photoRGB[0], photoRGB[1], photoRGB[2])

            # Convert from RGB to Lab Color Space   
            color1_lab = convert_color(color1_rgb, LabColor)

            # Find the color difference
            delta_e = delta_e_cie2000(color1_lab, color2_lab)

            if delta_e < delta['amount']:
                delta['amount'] = delta_e
                delta['path'] = photo['fileName']
        
        return delta['path']
    
    def deltaCIEOptimized(self, inputColor, cap=0):
        color2_rgb = sRGBColor(inputColor[0], inputColor[1], inputColor[2])
        # Convert from RGB to Lab Color Space
        color2_lab = convert_color(color2_rgb, LabColor)

        delta = {
            "path": "none",
            "amount": 10000000
        }

        enumPhotos = enumerate(self.photos)

        for i, photo in enumPhotos:
            photoRGB = eval(photo['dominantColor'])
            color1_rgb = sRGBColor(photoRGB[0], photoRGB[1], photoRGB[2])

            # Convert from RGB to Lab Color Space   
            color1_lab = convert_color(color1_rgb, LabColor)

            # Find the color difference
            delta_e = delta_e_cie2000(color1_lab, color2_lab)

            if delta_e < delta['amount']:
                delta['amount'] = delta_e
                delta['path'] = photo['fileName']
                delta['index'] = i
        
        # self.lookup[inputColor[0]][inputColor[1]][inputColor[2]][0] = delta['index']
        del self.photos[delta['index']]
        return delta['path']

def getDominantColorImage(img, pixelLength):
    img = img.resize((1,1), resample=0)
    img = img.resize((pixelLength, pixelLength))
    return img


def getDominantColorFromImage(img):
    img.resize((1,1), resample=0)
    return img.getpixel((0,0))


## 0.2 Masking Class

In [ ]:
# This class helps determine masked areas, user-determined areas of interest demarcated in 
# some unique color, to make some visual separation between it and less relevant regions
# of the image. 

class Mask:
    # Takes in the path of the input/target photo (unused), the masked PIL photo, and 
    # a tuple representing the mask color used (pink (255,0,200), for example)
    def __init__(self, targetPhotoPath, maskPhoto, maskColorRGB):
        self.maskPhoto = maskPhoto #Image.open(maskPhotoPath)
        self.maskColorRGB = maskColorRGB
    
    # Returns whether a particular region of the photo contains a mask (binary)
    def isMasked(self, cropBound):
        croppedMaskPhoto = self.maskPhoto.crop(cropBound)
        maskDominance = self.computeMaskDominance(croppedMaskPhoto, self.maskColorRGB)
        if maskDominance >= 0.25:
            return True
        return False

    def computeMaskDominance(self, croppedMaskPhoto, maskColorRGB):
        cmp = croppedMaskPhoto.convert('RGB')
        na = np.array(cmp)
        colors, counts = np.unique(na.reshape(-1,3), axis=0, return_counts=1)
        for i in range(len(colors)):
            if(all(colors[i] == np.array(maskColorRGB))):
                return (counts[i]/np.sum(counts))
        return 0


# FULLMASK = Image.open(INPUT_MASK_PATH)
# FULLMASK = FULLMASK.crop((0,0,750,950))
# FULLMASK.show()
# M = Mask(INPUT_IMAGE_PATH,FULLMASK,  (255,0,200))
# M.isMasked((0,0,750,950))

# Step 1: Index Album 
_You only need to run this once, or whenever your album changes._

In [259]:
def indexPhotoFolder(path, indexFileName, lim=None):
    photoPaths = glob.glob(path)

    dominant_indexed = []

    for i in range(len(photoPaths[0:lim])):
        photo = Image.open(photoPaths[i])
        dominantColor = str(getDominantColorFromImage(photo))
        dominant_indexed.append({"fileName":str(photoPaths[i]),"dominantColor":dominantColor})
        
        # print("Photos Indexed: {}/{}".format(i+1, len(photoPaths[0:lim])) )

    f = open(indexFileName, "w")
    f.write(json.dumps(dominant_indexed))
    f.close()
    print("Done indexing photo folder!")
    return 

# Run this to index dominant color from album
indexPhotoFolder(ALBUM_FOLDER_PATH + ALBUM_PHOTO_EXTENSION, ALBUM_FOLDER_INDEX)

Done indexing photo folder!


____________________________________________________________

# Step 2: Compute Image

In [262]:
selectedStrategyIndex = 3 # 3 works best
downSizeFactor = 5 # make photo 5x smaller
MASKED_BLEND = 0.8 # 1 = fully show masked photo
UNMASKED_BLEND = 1 # 1 = fully show unmasked regions

strategies = [
    "dominantFromTarget", 
    "dominantFromAlbum", 
    "dominantFromAlbumWithBlend", 
    "dominantFromAlbumWithBlendWithoutReplacement",
]

# Downsize image to make processing a bit easier. 
selectedPhoto = Image.open(INPUT_IMAGE_PATH)
maskedPhoto = Image.open(INPUT_MASK_PATH)
initialPhotoWidth, initialPhotoHeight = selectedPhoto.size
selectedPhoto = selectedPhoto.resize((int(initialPhotoWidth / downSizeFactor), int(initialPhotoHeight / downSizeFactor)))
maskedPhoto = maskedPhoto.resize((int(initialPhotoWidth / downSizeFactor), int(initialPhotoHeight / downSizeFactor)))
initialPhotoWidth, initialPhotoHeight = selectedPhoto.size

# Determine scale of new image
scaleFactor = int(np.floor(initialPhotoWidth/numberSubPhotosHorizontal))
numberSubPhotosVertical = int(np.floor(initialPhotoHeight / scaleFactor))
finalPhotoWidth = int(subPhotoPixelLength * numberSubPhotosHorizontal)
finalPhotoHeight = int(subPhotoPixelLength  * numberSubPhotosVertical)

finalPhoto = Image.new("RGB", (finalPhotoWidth, finalPhotoHeight))

def resizeImageToSquare(photoPath, dimension):
    imga = Image.open(photoPath)

    # Resize
    width, height = imga.size
    scaleFactor = 1
    # Landscape
    if width > height: 
        scaleFactor = height / dimension
    # Portrait
    if width <= height: 
        scaleFactor = width / dimension
    imga = imga.resize((int(width / scaleFactor), int(height / scaleFactor)))
    
    # Crop
    width, height = imga.size
    # Landsacpe
    if width > height: 
        inset = int((width - dimension ) / 2)
        imga = imga.crop((inset, 0, inset + dimension, dimension ))
    # Portrait
    if width <= height: 
        inset = int((height - dimension ) / 2)
        imga = imga.crop((0, inset, dimension, inset + dimension ))

    return imga

def formatAlbumPhoto(photoPath, pixelLength, blend=False, alpha=0.5, dominantPhoto=False):
    photo = resizeImageToSquare(photoPath, pixelLength)
    if blend:
        try:
            return Image.blend(photo, dominantPhoto, alpha)
        except ValueError:
            print("ERROR")
            photo.show()
            dominantPhoto.show()
    return photo


def findBestMatchPhoto(cropPhoto, pixelLength, isMasked=False):
    blendAmount = MASKED_BLEND if isMasked else UNMASKED_BLEND
    photoToBlend = cropPhoto.resize((pixelLength, pixelLength)) if isMasked else getDominantColorImage(cropPhoto, pixelLength)

    # Strat 0: Just get dominant color from original target image
    if (strategies[selectedStrategyIndex] == 'dominantFromTarget'):
        dominant_color = getDominantColorFromImage(cropPhoto)
        return Image.new('RGB', (pixelLength, pixelLength), dominant_color)
    
    # Strat 1: Sub in the most visually similar image from album
    if (strategies[selectedStrategyIndex] == "dominantFromAlbum"):
        dominantColorFromTarget = getDominantColorFromImage(cropPhoto)
        photoPath = D.deltaCIE(dominantColorFromTarget)
        photo = formatAlbumPhoto(photoPath, pixelLength)
        return photo
    
    # Strat 2: Blend in most visually similar image from album, with replacement
    # i.e. images can be reused. 
    if (strategies[selectedStrategyIndex] == "dominantFromAlbumWithBlend"):
        dominantColorFromTarget = getDominantColorFromImage(cropPhoto)
        photoPath = D.deltaCIE(dominantColorFromTarget)
        photo = formatAlbumPhoto(photoPath, pixelLength, True, blendAmount, photoToBlend)
        return photo
    
    # Strat 3: Blend in most visually similar image from album, no replacement
    # i.e. images will not be reused. 
    # note you MUST not run out of photos! 
    if (strategies[selectedStrategyIndex] == "dominantFromAlbumWithBlendWithoutReplacement"):
        dominantColorFromTarget = getDominantColorFromImage(cropPhoto)
        photoPath = D.deltaCIEOptimized(dominantColorFromTarget)
        photo = formatAlbumPhoto(photoPath, pixelLength, True, blendAmount, photoToBlend)
        return photo

D = DistanceHeuristics(ALBUM_FOLDER_PATH + 'indexed.txt')
M = Mask(INPUT_IMAGE_PATH, maskedPhoto, MASK_COLOR_RGB)

# Iterate over horizontal subphotos
for i in range(numberSubPhotosHorizontal):
    # Iterate over vertical subphotos
    for j in range(numberSubPhotosVertical):
        cropBound = (i * scaleFactor, j * scaleFactor, i * scaleFactor + scaleFactor, j * scaleFactor + scaleFactor)
        isMasked = M.isMasked(cropBound)
        selectedPhotoCrop = selectedPhoto.crop(cropBound)
        bestMatchPhoto = findBestMatchPhoto(selectedPhotoCrop, subPhotoPixelLength, isMasked=isMasked)
        finalPhoto.paste(bestMatchPhoto, box=(i * subPhotoPixelLength, j * subPhotoPixelLength))

        print(f"Completed {i}/{numberSubPhotosHorizontal}, {j}/{numberSubPhotosVertical}", end='\r')

finalPhoto.save(OUTPUT_IMAGE_PATH)
print(f"Success! Saved to {OUTPUT_IMAGE_PATH}")


Photos index loaded.
Image saved to data/outputimages/spidermanoutput-1.jpg


# Bonus: Bing Image Downloader
For quickly downloading new datasets of photos! 

In [ ]:
from bing_image_downloader import downloader
query_string = 'lebron james'
downloader.download(query_string, limit=150,  output_dir='data/albums', adult_filter_off=True, force_replace=False, timeout=60, verbose=False)